In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [5]:
import wandb

In [7]:
wandb.init(project="CIFAR Classification Example(Train)")

In [9]:
wandb.run.name = "First wandb"
wandb.run.save()

wandb: WARNING Calling wandb.run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


True

In [10]:
batch_size = 4
learning_rate = 0.001
epochs = 5

In [11]:
args = {
    "learning_rate": learning_rate,
    "epochs": epochs,
    "batch_size": batch_size
}

In [12]:
wandb.config.update(args)

## Load and normalize CIFAR10

In [13]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [14]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

100%|███████████████████████████████████████████████████████████████| 170498071/170498071 [00:23<00:00, 7136592.67it/s]


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified


## Define a CNN

In [15]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print("device:", device)

device: cuda:0


In [16]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) 
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [17]:
net = Net().to(device)

## Define a Loss Function and optimizer

In [18]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters())

## Train the network

In [20]:
for epoch in range(epochs): # loop ovner the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistic
        running_loss += loss.item()
        if i % 2000 == 1999: # print every 2000 mini batches
            print(f"[{epochs + 1}, {i + 1: 5d}] loss: {running_loss / 2000:.3f}")
            wandb.log({"Training loss": running_loss / 2000})
            runningLoss = 0.0

[6,  2000] loss: 1.853
[6,  4000] loss: 3.453
[6,  6000] loss: 4.945
[6,  8000] loss: 6.389
[6,  10000] loss: 7.778
[6,  12000] loss: 9.142
[6,  2000] loss: 1.292
[6,  4000] loss: 2.569
[6,  6000] loss: 3.845
[6,  8000] loss: 5.095
[6,  10000] loss: 6.338
[6,  12000] loss: 7.583
[6,  2000] loss: 1.168
[6,  4000] loss: 2.337
[6,  6000] loss: 3.520
[6,  8000] loss: 4.690
[6,  10000] loss: 5.848
[6,  12000] loss: 7.002
[6,  2000] loss: 1.085
[6,  4000] loss: 2.205
[6,  6000] loss: 3.298
[6,  8000] loss: 4.418
[6,  10000] loss: 5.551
[6,  12000] loss: 6.650
[6,  2000] loss: 1.027
[6,  4000] loss: 2.086
[6,  6000] loss: 3.153
[6,  8000] loss: 4.230
[6,  10000] loss: 5.329
[6,  12000] loss: 6.404


In [21]:
PATH = '.cifar_net.pth'
torch.save(net.state_dict(), PATH)
print("Finished Training")

Finished Training
